In [1]:
CODE_ROOT = pwd() * "/"

include(CODE_ROOT * "mode_forward/time_steppers.jl")
include(CODE_ROOT * "mode_init/MPAS_Ocean.jl")
include(CODE_ROOT  * "visualization.jl")

vertexHeatMapMesh (generic function with 1 method)

In [2]:
import CUDA

In [3]:
const MAX_THREADS_PER_BLOCK = CUDA.attribute(
        CUDA.CuDevice(0), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK,
)

1024

# Load Mesh From File

In [4]:
mpas32 = MPAS_Ocean{Float32}(CODE_ROOT * "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                "base_mesh.nc", "mesh.nc", periodicity="Periodic")
typeof(mpas32)

MPAS_Ocean{Float32}

In [5]:
mpas64 = MPAS_Ocean{Float64}(CODE_ROOT * "MPAS_O_Shallow_Water/Mesh+Initial_Condition+Registry_Files/Periodic",
                "base_mesh.nc", "mesh.nc", periodicity="Periodic")
typeof(mpas32)

MPAS_Ocean{Float32}

In [6]:
mpasOcean = mpas64
typeof(mpasOcean)

MPAS_Ocean{Float64}

# Define Initial Condition

In [7]:
function gaussianInit!(mpasOcean; sx=1/3, sy=1/3, mx=1/2, my=1/2)
    sigmax = mpasOcean.lX*sx
    sigmay = mpasOcean.lY*sy
    
    mux = mpasOcean.lX*mx
    muy = mpasOcean.lY*my
    
    for iCell in 1:mpasOcean.nCells
        mpasOcean.sshCurrent[iCell] = exp( - (mpasOcean.xCell[iCell] - mux)^2 / sigmax^2 - (mpasOcean.yCell[iCell] - muy)^2 / sigmay^2 )
    end
    
    mpasOcean.normalVelocityCurrent .= 0
    
    return nothing
end

gaussianInit! (generic function with 1 method)

In [8]:
function planeWaveInit!(mpasOcean; rkx = 1, rky = 1)
    kx = rkx * 2 * pi / mpasOcean.lX
    ky = rky * 2 * pi / mpasOcean.lY
    
    for iCell in 1:mpasOcean.nCells
        mpasOcean.sshCurrent[iCell] = cos( kx * mpasOcean.xCell[iCell] + ky * mpasOcean.yCell[iCell] )
    end
    
    mpasOcean.normalVelocityCurrent .= 0
    
    mpasOcean.sshTendency[:] .= mpasOcean.sshCurrent[:]
    mpasOcean.normalVelocityTendency[:] .= mpasOcean.normalVelocityCurrent[:]
    
    return nothing
end

planeWaveInit! (generic function with 1 method)

In [9]:
moveArrays!(mpasOcean, Array)

planeWaveInit!(mpasOcean) # could also do gaussianInit!

# Benchmark GPU vs CPU

In [10]:
using BenchmarkTools

In [11]:
moveArrays!(mpasOcean, Array)
gaussianInit!(mpasOcean)
# save gaussian initial condition so it doesn't have to be recalculated every time
initialSSH = mpasOcean.sshCurrent[:]
initialNormalVelocity = mpasOcean.normalVelocityCurrent[:]

7500-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
# benchmark time it takes to do a timestep on the CPU
moveArrays!(mpas64, Array)
mpas64.sshCurrent[:] = initialSSH[:]
mpas64.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark forward_backward_step!(mpas64)

BenchmarkTools.Trial: 256 samples with 1 evaluation.
 Range (min … max):  18.344 ms … 24.109 ms  ┊ GC (min … max): 0.00% … 10.48%
 Time  (median):     18.745 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.552 ms ±  1.234 ms  ┊ GC (mean ± σ):  4.56% ±  5.37%

  ▅█▅▃▂▃                         ▄▆▃▃ ▁▁                       
  ███████▆▄▄▄▇▁▁▁▁▄█▄▆▆▆▁▁▁▁▁▁▁▁▄████▇██▇▆▁▁▁▁▁▁▄▄▁▁▁▁▄▁▄▆▆▁▄ ▆
  18.3 ms      Histogram: log(frequency) by time      22.6 ms <

 Memory estimate: 17.05 MiB, allocs estimate: 955195.

In [13]:
meantimecpu = mean(cpubench.times)

1.95517237578125e7

In [14]:
sqrt(mean((cpubench.times .- meantimecpu).^2))

1.2318661158168875e6

In [15]:
# benchmark time it takes to do a timestep on the GPU
mpas64.sshCurrent[:] = initialSSH[:]
mpas64.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpas64, CUDA.CuArray)
gpubench = @benchmark forward_backward_step_cuda!(mpas64) samples=10000

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  17.461 μs …  4.168 ms  ┊ GC (min … max): 0.00% … 99.00%
 Time  (median):     40.845 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.811 μs ± 58.648 μs  ┊ GC (mean ± σ):  2.07% ±  1.40%

                                                        ▆█     
  ▂▃▃▃▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▅███▃▂ ▂
  17.5 μs         Histogram: frequency by time        42.5 μs <

 Memory estimate: 8.97 KiB, allocs estimate: 178.

In [16]:
meantimegpu = mean(gpubench.times)

39810.6105

In [17]:
sqrt(mean((gpubench.times .- meantimegpu).^2))

58645.404371583885

In [18]:
meantimecpu/meantimegpu

491.1184107014008

That's an incredible speedup!

In [16]:
include("mode_init/MPAS_Ocean_CUDA.jl")

update_ssh_by_tendency_cuda_kernel! (generic function with 1 method)

In [18]:
mpas64.sshCurrent[:] = initialSSH[:]
mpas64.normalVelocityCurrent[:] = initialNormalVelocity[:]
mpasOceanCuda = MPAS_Ocean_CUDA(mpas64)

MPAS_Ocean_CUDA([0.9892723329629883, 0.9631625667976582, 0.9218631515885005, 0.8660254037844387, 0.7965299180241964, 0.7144726796328034, 0.6211477802783104, 0.5180270093731303, 0.4067366430758004, 0.28903179694447184  …  0.38751558645210105, 0.4999999999999991, 0.6045991148623748, 0.6996633405133649, 0.783693457325839, 0.8553642601605055, 0.9135454576426004, 0.9573194975320672, 0.9859960370705048, 0.9991228300988583], [0.013670705776345467, 0.016249388527358702, 0.019175919112927212, 0.022467173289690667, 0.02613447382456567, 0.030182288504651218, 0.03460697595652069, 0.03939564507021855, 0.044525198128071264, 0.0499616284090268  …  0.04451184257206646, 0.039525865444392724, 0.03484658980774021, 0.030500872171217483, 0.026505580056142567, 0.022868382483771488, 0.019588747201293746, 0.01665907776530237, 0.014065926028455205, 0.011791221087892932], [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0  …  1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1

In [19]:
gpubench2 = @benchmark forward_backward_step_cuda!(mpasOceanCuda) samples=10000

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):   5.500 μs … 469.126 μs  ┊ GC (min … max): 0.00% … 91.27%
 Time  (median):     50.424 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.268 μs ±   5.518 μs  ┊ GC (mean ± σ):  0.09% ±  0.91%

                                                           ▇█▁ ▁
  █▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▃▁▁▁▁▁▁▁▃▃▅▅▄▅▅▅▅███ █
  5.5 μs        Histogram: log(frequency) by time      51.9 μs <

 Memory estimate: 1.36 KiB, allocs estimate: 17.

## whole time step

### double precision

In [121]:
function func1!(mpasOcean)
    mpasOcean.normalVelocityTendency[:] .= convert(eltype(mpasOcean.normalVelocityTendency), 0)

    for iEdge in 1:mpasOcean.nEdges
        # gravity term: take gradient of sshCurrent across edge
        cell1Index, cell2Index = 1, 2 #mpasOcean.cellsOnEdge[:,iEdge]
        
#         if cell1Index != 0 && cell2Index != 0
            mpasOcean.normalVelocityTendency[iEdge] = mpasOcean.gravity * ( mpasOcean.sshCurrent[cell1Index] - mpasOcean.sshCurrent[cell2Index] ) / mpasOcean.dcEdge[iEdge]
#         end
        
#         # coriolis term
#         for i in 1:mpasOcean.nEdgesOnEdge[iEdge]
#             eoe = mpasOcean.edgesOnEdge[i,iEdge]
            
#             if eoe != 0
#                     mpasOcean.normalVelocityTendency[iEdge] += mpasOcean.weightsOnEdge[i,iEdge] * mpasOcean.normalVelocityCurrent[eoe] * mpasOcean.fEdge[eoe]
#             end
#         end
    end
end

func1! (generic function with 1 method)

In [119]:
mpasOcean.cellsOnEdge

2×7500 Matrix{Int32}:
 2497  2489  2488  2487  2485  2484  …  1211  2300  1212  1212  1979  1213
 2498  2490  2489  2488  2486  2485     1212  2301  1262  1261  2029  1214

In [122]:
function func1dot!(mpasOcean)
    mpasOcean.normalVelocityTendency[:] .= convert(eltype(mpasOcean.normalVelocityTendency), 0)

        # gravity term: take gradient of sshCurrent across edge
        cell1Index, cell2Index = 1, 2 #mpasOcean.cellsOnEdge[:,iEdge]
        
#         if cell1Index != 0 && cell2Index != 0
    mpasOcean.normalVelocityTendency .= mpasOcean.gravity .* ( mpasOcean.sshCurrent[mpasOcean.cellsOnEdge[1,:]] .- mpasOcean.sshCurrent[mpasOcean.cellsOnEdge[2,:]] ) ./ mpasOcean.dcEdge
#         end
        
#         # coriolis term
#         for i in 1:mpasOcean.nEdgesOnEdge[iEdge]
#             eoe = mpasOcean.edgesOnEdge[i,iEdge]
            
#             if eoe != 0
#                     mpasOcean.normalVelocityTendency[iEdge] += mpasOcean.weightsOnEdge[i,iEdge] * mpasOcean.normalVelocityCurrent[eoe] * mpasOcean.fEdge[eoe]
    #             end
                    mpasOcean.normalVelocityTendency .+= mpasOcean.weightsOnEdge[:,:] .* mpasOcean.normalVelocityCurrent[mpasOcean.edgesOnEdge]
#         end
end

func1dot! (generic function with 1 method)

In [76]:
mpasOcean.sshCurrent[mpasOcean.cellsOnEdge]

2×7500 Matrix{Float32}:
 634.263  775.076  530.594  414.946  457.528  …  -758.347    6.8425  -862.425
 816.749  779.639  775.076  530.594  305.573     -351.137  417.413   -822.532

In [123]:
sampsize = 10000

10000

In [124]:
mpasOcean = mpas32
@time for i in 1:sampsize
    func1dot!(mpasOcean)
end

  0.500486 seconds (1.89 M allocations: 1.208 GiB, 10.23% gc time)


In [125]:
mpasOcean = mpas32
@time for i in 1:sampsize
    func1!(mpasOcean)
end

 11.642514 seconds (741.55 M allocations: 12.228 GiB, 6.47% gc time)


In [126]:
mpasOcean = mpas64
@time for i in 1:sampsize
    func1dot!(mpasOcean)
end

  0.546731 seconds (1.89 M allocations: 1.767 GiB, 10.42% gc time)


In [127]:
mpasOcean = mpas64
@time for i in 1:sampsize
    func1!(mpasOcean)
end

 10.680777 seconds (741.55 M allocations: 12.228 GiB, 7.06% gc time)


In [115]:
0.32/0.26

1.2307692307692308

In [117]:
11.43/10.51

1.087535680304472

In [73]:
using DelimitedFiles

In [48]:
open(CODE_ROOT * "output/gpubenchmark.txt", "w") do file
    writedlm(file, [string(gpubench.params), gpubench.times, gpubench.gctimes, gpubench.memory, gpubench.allocs])
end

In [74]:
open(CODE_ROOT * "output/cpubenchmark.txt", "w") do file
    writedlm(file, [string(cpubench.params), cpubench.times, cpubench.gctimes, cpubench.memory, cpubench.allocs])
end

In [91]:
println("CUDA kernel evaluation is $(mean(cpubench.times)/mean(gpubench.times)) times faster than the CPU evaluation on average")

CUDA kernel evaluation is 603.8124471873369 times faster than the CPU evaluation on average


### single precision

In [16]:
# benchmark time it takes to do a timestep on the GPU
mpas32.sshCurrent[:] = initialSSH[:]
mpas32.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpas32, CUDA.CuArray)
gpubench = @benchmark forward_backward_step_cuda!(mpas32)

BechmarkTools.Trial: 10000 samples with 1 evaluations.
 Range (min … max):  17.796 μs …  4.374 ms  ┊ GC (min … max): 0.00% … 98.98%
 Time  (median):     41.169 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.691 μs ± 62.356 μs  ┊ GC (mean ± σ):  2.12% ±  1.40%

  ▁▄▃                                           ▂▅██▄▁        ▂
  ████▆▅▄▅▁▃▃▁▄▄▁▁▄▅▄▄▅▄▄▄▁▁▁▁▄▁▄▃▁▄▃▁▄▄▁▃▄▃▆▆▇████████▇▆▅▄▆▅ █
  17.8 μs      Histogram: log(frequency) by time      46.1 μs <

 Memory estimate: 8.97 KiB, allocs estimate: 178.

In [21]:
# benchmark time it takes to do a timestep on the CPU
moveArrays!(mpas32, Array)
mpas32.sshCurrent[:] = initialSSH[:]
mpas32.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark forward_backward_step!(mpas32)

BechmarkTools.Trial: 210 samples with 1 evaluations.
 Range (min … max):  22.591 ms … 27.362 ms  ┊ GC (min … max): 0.00% … 10.67%
 Time  (median):     22.699 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.853 ms ±  1.469 ms  ┊ GC (mean ± σ):  4.82% ±  5.61%

  █▃                                                           
  ██▄▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃█▇▄▄▂▃▂▃▂ ▂
  22.6 ms         Histogram: frequency by time          26 ms <

 Memory estimate: 17.28 MiB, allocs estimate: 970201.

In [18]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 522.0680016610622 times faster than the CPU evaluation on average"

## double and single precision appear to result in the same speed
This must be some mistake of the compiler converting them to the same type under the hood. It could be accelerated further using single-precision, possibly up to 8 times (current GPU hardware runs much faster on single precision.)

In [92]:
mpasOcean = mpas32
moveArrays!(mpasOcean, Array)
for fieldname in fieldnames(MPAS_Ocean)
    println(fieldname, "\t\t\t", typeof(getfield(mpasOcean, fieldname)))
end

boundaryConditions			Vector{BoundaryCondition}
sshCurrent			Vector{Float32}
sshTendency			Vector{Float32}
normalVelocityCurrent			Vector{Float32}
normalVelocityTendency			Vector{Float32}
bottomDepth			Vector{Float32}
gravity			Float32
dt			Float32
nCells			Int64
cellsOnCell			Matrix{Int32}
edgesOnCell			Matrix{Int32}
verticesOnCell			Matrix{Int32}
kiteIndexOnCell			Matrix{Int64}
nEdgesOnCell			Vector{Int32}
edgeSignOnCell			Matrix{Int8}
latCell			Vector{Float32}
lonCell			Vector{Float32}
xCell			Vector{Float32}
yCell			Vector{Float32}
areaCell			Vector{Float32}
fCell			Vector{Float32}
maxLevelCell			Vector{Int64}
gridSpacing			Vector{Float32}
boundaryCell			Matrix{Int8}
nEdges			Int64
cellsOnEdge			Matrix{Int32}
edgesOnEdge			Matrix{Int32}
verticesOnEdge			Matrix{Int32}
nEdgesOnEdge			Vector{Int32}
xEdge			Vector{Float32}
yEdge			Vector{Float32}
dvEdge			Vector{Float32}
dcEdge			Vector{Float32}
fEdge			Vector{Float32}
angleEdge			Vector{Float32}
weightsOnEdge			Matrix{Float32}
maxLevel

In [83]:
function just_show_precision(array)
    CUDA.@cushow typeof(array)
    CUDA.@cushow eltype(array)
    square = array .* array
    CUDA.@cushow typeof(square)
    CUDA.@cushow eltype(square)
    return nothing
end

just_show_precision (generic function with 1 method)

In [84]:
moveArrays!(mpasOcean, CUDA.CuArray, Float32)
CUDA.@cuda threads=500 blocks=20 just_show_precision(mpasOcean.sshCurrent)

┌ Error: @cuprint does not support non-literal strings
└ @ CUDA /home/rrs/.julia/packages/CUDA/02Kjq/src/device/intrinsics/output.jl:166
┌ Error: @cuprint does not support non-literal strings
└ @ CUDA /home/rrs/.julia/packages/CUDA/02Kjq/src/device/intrinsics/output.jl:166
┌ Error: @cuprint does not support non-literal strings
└ @ CUDA /home/rrs/.julia/packages/CUDA/02Kjq/src/device/intrinsics/output.jl:166
┌ Error: @cuprint does not support non-literal strings
└ @ CUDA /home/rrs/.julia/packages/CUDA/02Kjq/src/device/intrinsics/output.jl:166


LoadError: InvalidIRError: compiling kernel just_show_precision(CUDA.CuDeviceVector{Float32, 1}) resulted in invalid LLVM IR
Reason: unsupported call through a literal pointer (call to )
Stacktrace:
 [1] [0m[1m_string_n[22m
[90m   @ [39m[90m./strings/[39m[90;4mstring.jl:74[0m
 [2] [0m[1mStringVector[22m
[90m   @ [39m[90m./[39m[90;4miobuffer.jl:31[0m
 [3] [0m[1m#IOBuffer#361[22m
[90m   @ [39m[90m./[39m[90;4miobuffer.jl:114[0m
 [4] [0m[1mprint_to_string[22m
[90m   @ [39m[90m./strings/[39m[90;4mio.jl:133[0m
 [5] [0m[1mmultiple call sites[22m
[90m   @ [39m[90;4munknown:0[0m
Reason: unsupported call through a literal pointer (call to )
Stacktrace:
 [1] [0m[1munsafe_wrap[22m
[90m   @ [39m[90m./strings/[39m[90;4mstring.jl:85[0m
 [2] [0m[1mStringVector[22m
[90m   @ [39m[90m./[39m[90;4miobuffer.jl:31[0m
 [3] [0m[1m#IOBuffer#361[22m
[90m   @ [39m[90m./[39m[90;4miobuffer.jl:114[0m
 [4] [0m[1mprint_to_string[22m
[90m   @ [3

### normal velocty tendency calculation only

In [21]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_normal_velocity_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.501 μs … 743.442 μs  ┊ GC (min … max): 0.00% … 97.60%
 Time  (median):     20.223 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   20.198 μs ±  15.009 μs  ┊ GC (mean ± σ):  1.43% ±  1.95%

  ▃                                                   ▄█       ▁
  ██▄▃▁▁▄▄▄▁▄▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▃▁▁▁▃▁▁▁▁▃▄▄▅▇▇▆▆▆▆██▆▆▆▆▆▇ █
  4.5 μs        Histogram: log(frequency) by time      22.2 μs <

 Memory estimate: 3.08 KiB, allocs estimate: 57.

In [22]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark calculate_normal_velocity_tendency!(mpasOcean)

BechmarkTools.Trial: 307 samples with 1 evaluations.
 Range (min … max):  14.158 ms … 21.202 ms  ┊ GC (min … max): 0.00% … 19.17%
 Time  (median):     15.531 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.294 ms ±  1.730 ms  ┊ GC (mean ± σ):  6.43% ±  9.40%

          ▁█  ▄▆                                               
  ▄▃▄▃▅▄▄▆██▅████▆▅▅▁▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁▁▁▃▃▃▃▄▃▄▃▆▅▅▃▄▂▃▂▂▄▄▂▂▂ ▃
  14.2 ms         Histogram: frequency by time        20.3 ms <

 Memory estimate: 13.24 MiB, allocs estimate: 725293.

In [23]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 806.7136301657602 times faster than the CPU evaluation on average"

### experimenting with making method more effecient

In [13]:
function calculate_normal_velocity_tendency_cuda2!(mpasOcean::MPAS_Ocean)
    
    CUDA.@cuda blocks=cld(mpasOcean.nEdges, 1024) threads=1024 maxregs=64 calculate_normal_velocity_tendency_cuda_kernel2!(
                                                                        mpasOcean.nEdges,
                                                                        mpasOcean.normalVelocityTendency,
                                                                        mpasOcean.normalVelocityCurrent,
                                                                        mpasOcean.sshCurrent,
                                                                        mpasOcean.cellsOnEdge,
                                                                        mpasOcean.nEdgesOnEdge,
                                                                        mpasOcean.edgesOnEdge,
                                                                        mpasOcean.weightsOnEdge,
                                                                        mpasOcean.fEdge,
                                                                        mpasOcean.dcEdge,
                                                                        mpasOcean.gravity)
end

calculate_normal_velocity_tendency_cuda2! (generic function with 1 method)

In [14]:
function calculate_normal_velocity_tendency_cuda_kernel2!(nEdges,
                                                          normalVelocityTendency,
                                                          normalVelocity,
                                                          ssh,
                                                          cellsOnEdge,
                                                          nEdgesOnEdge,
                                                          edgesOnEdge,
                                                          weightsOnEdge,
                                                          fEdge,
                                                          dcEdge,
                                                          gravity)
    
    iEdge = (CUDA.blockIdx().x - 1) * CUDA.blockDim().x + CUDA.threadIdx().x
    if iEdge <= nEdges
        # read global memory into thread registers
        cell1 = cellsOnEdge[1,iEdge]
        ssh1 = ssh[cell1]
        cell2 = cellsOnEdge[2,iEdge]
        ssh2 = ssh[cell2]
        dc = dcEdge[iEdge]
        neoe = nEdgesOnEdge[iEdge]
        
#         eoes = edgesOnEdge[iEdge]
        
        
        
        # gravity term: take gradient of ssh across edge
        tendency = gravity * ( ssh1 - ssh2 ) / dc

        # coriolis term: combine norm. vel. of surrounding edges to approx. tangential vel.
        for i = 1:neoe
            eoe = edgesOnEdge[i,iEdge]
            tendency += weightsOnEdge[i,iEdge] * normalVelocity[eoe] * fEdge[eoe]
        end

        normalVelocityTendency[iEdge] = tendency
    end
    return
end

calculate_normal_velocity_tendency_cuda_kernel2! (generic function with 1 method)

In [15]:
function calculate_ssh_tendency_cuda_kernel2!(nCells,
                                              sshTendency,
                                              ssh,
                                              normalVelocity,
                                              bottomDepth,
                                              nEdgesOnCell,
                                              edgesOnCell,
                                              cellsOnCell,
                                              areaCell,
                                              edgeSignOnCell,
                                              dvEdge)
    
    iCell = (CUDA.blockIdx().x - 1) * CUDA.blockDim().x + CUDA.threadIdx().x
    if iCell <= nCells
        
        sshTendency[iCell] = 0
        
        # sum flux through each edge of cell
        for i in 1:nEdgesOnCell[iCell]
            edgeID = edgesOnCell[i,iCell]
            neighborCellID = cellsOnCell[i,iCell]

            if neighborCellID !== 0
                depth = ( bottomDepth[neighborCellID] + bottomDepth[iCell] ) / 2.0
            else
                depth = bottomDepth[iCell]
            end
            
            flux = depth * normalVelocity[edgeID]
            sshTendency[iCell] += flux * edgeSignOnCell[iCell,i] * dvEdge[edgeID] / areaCell[iCell]
        end
        
    end
    return
end


calculate_ssh_tendency_cuda_kernel2! (generic function with 1 method)

In [16]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_normal_velocity_tendency_cuda2!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.347 μs … 732.013 μs  ┊ GC (min … max): 0.00% … 97.58%
 Time  (median):     16.960 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.270 μs ±  14.179 μs  ┊ GC (mean ± σ):  1.50% ±  1.95%

  ▃                ▂█                                          ▁
  █▅▄▆▄▄▃▄▄▅▅▅▅▇▇██████▇▇▅▅▄▄▄▄▃▄▁▄▁▃▄▄▅▄▃▃▃▃▃▄▄▅▅▅▄▁▁▁▁▁▃▄▅▅▃ █
  4.35 μs       Histogram: log(frequency) by time      45.9 μs <

 Memory estimate: 3.08 KiB, allocs estimate: 57.

17 microseconds rather than 20, a slight improvement

### ssh tendency calculation only

In [25]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark calculate_ssh_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   4.374 μs … 760.519 μs  ┊ GC (min … max): 0.00% … 96.73%
 Time  (median):     27.183 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.210 μs ±  14.107 μs  ┊ GC (mean ± σ):  0.81% ±  1.69%

  ▃                         █                                  ▁
  █▄▄▃▁▃▃▁▁▁▁▃▁▁▁▁▁▁▃▃▁▃▄▇████▇▄▄▁▁▁▁▁▁▁▁▁▄▁▁▃▄▄▃▁▁▁▁▁▁▄▁▅▆▄▃▆ █
  4.37 μs       Histogram: log(frequency) by time      56.2 μs <

 Memory estimate: 3.11 KiB, allocs estimate: 55.

In [26]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark calculate_ssh_tendency!(mpasOcean)

BechmarkTools.Trial: 1283 samples with 1 evaluations.
 Range (min … max):  3.539 ms …   7.596 ms  ┊ GC (min … max): 0.00% … 41.13%
 Time  (median):     3.672 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.895 ms ± 746.883 μs  ┊ GC (mean ± σ):  5.31% ± 11.34%

  ▂▆█▄▄▃                                                ▁      
  ██████▇▅▆▆▁▁▄▄▄▄▄▁▁▅▆▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▇███▇▇▆ █
  3.54 ms      Histogram: log(frequency) by time      6.69 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 229892.

In [27]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 143.1461720325683 times faster than the CPU evaluation on average"

### update norm vel

In [28]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark update_normal_velocity_by_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.152 μs … 735.304 μs  ┊ GC (min … max): 0.00% … 99.27%
 Time  (median):     3.337 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.535 μs ±  10.279 μs  ┊ GC (mean ± σ):  4.10% ±  1.40%

     ██▇▅▁                                                     
  ▂▂▆█████▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  3.15 μs         Histogram: frequency by time        5.14 μs <

 Memory estimate: 1.39 KiB, allocs estimate: 33.

In [29]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark update_normal_velocity_by_tendency!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.454 μs …   5.509 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.545 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.580 μs ± 286.345 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▂█▁                                                        
  ▆████▄▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▂
  1.45 μs         Histogram: frequency by time        3.14 μs <

 Memory estimate: 80 bytes, allocs estimate: 3.

In [30]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 0.44711592195483957 times faster than the CPU evaluation on average"

### update ssh

In [31]:
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
moveArrays!(mpasOcean, CUDA.CuArray)
gpubench = @benchmark update_ssh_by_tendency_cuda!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.021 μs … 729.140 μs  ┊ GC (min … max): 0.00% … 99.33%
 Time  (median):     3.220 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.348 μs ±   7.266 μs  ┊ GC (mean ± σ):  2.16% ±  0.99%

     ▃██▆▄▁                                                    
  ▂▃▆██████▆▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  320 μs          Histogram: frequency by time        4.94 μs <

 Memory estimate: 1.39 KiB, allocs estimate: 33.

In [32]:
moveArrays!(mpasOcean, Array)
mpasOcean.sshCurrent[:] = initialSSH[:]
mpasOcean.normalVelocityCurrent[:] = initialNormalVelocity[:]
cpubench = @benchmark update_ssh_by_tendency!(mpasOcean)

BechmarkTools.Trial: 10000 samples with 162 evaluations.
 Range (min … max):  664.253 ns …  27.238 μs  ┊ GC (min … max): 0.00% … 97.05%
 Time  (median):     713.883 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   722.487 ns ± 371.018 ns  ┊ GC (mean ± σ):  0.72% ±  1.37%

           ▁▂▃▄█▇▃▂▁                                             
  ▂▂▃▄▄▆█▇▇█████████▇▇▆▅▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂ ▃
  664 ns           Histogram: frequency by time          885 ns <

 Memory estimate: 80 bytes, allocs estimate: 3.

In [33]:
"CUDA kernel evaluation is $((sum(cpubench.times)/length(cpubench.times)) / (sum(gpubench.times)/length(gpubench.times))) times faster than the CPU evaluation on average"

"CUDA kernel evaluation is 0.21581601461375574 times faster than the CPU evaluation on average"

the data copying step is actually 2 - 5 times slower than it is on the CPU! Memory is the bottleneck of the GPU